In [12]:
# #
import os

In [10]:
os.environ['JAVA_HOME'] = '~/Data-Engineering_text-to-speech_data-collection/env/lib/python3.8/site-packages'

In [11]:
import pyspark
import findspark
print(f'pyspark version: {pyspark.__version__}')
print(f'findspark version: {findspark.__version__}')


pyspark version: 3.3.0
findspark version: 2.0.1


In [ ]:
## ./env/lib/python3.8/site-packages

In [13]:
# importing lib
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import Catalog
import pyspark.sql.functions as psf
from pyspark.sql.functions import lit, col,sum,avg,max,first,min,mean
import pyspark.ml as Pipeline
import pyspark.ml.pipeline as pmpip
import pyspark.ml.param as pmparam
from pyspark.ml.param.shared import HasInputCol,HasOutputCol
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,FloatType
from pyspark.sql.functions import monotonically_increasing_id 
from pyspark.sql.functions import struct,array,lit
from pyspark.streaming import StreamingContext

In [14]:
# creating configuration
spark_conf = SparkConf()
spark_conf.setAll([
    ('spark.master', 'local'),
    ('spark.app.name', 'text-speech-model'),
    ('spark.submit.deployMode', 'client'),
    ('spark.ui.showConsoleProgress', 'true'),
    ('spark.eventLog.enabled', 'false'),
    ('spark.logConf', 'false'),
    ('diriver_port', '8887'),
    ('cores_max','4'),
    ('ui_port','4040'),
    ('spark.driver.host', 'localhost'),
    ('spark.driver.memory','4g')
])

In [9]:
 # Use your own conda python bin here
os.environ['PYSPARK_PYTHON'] = './spark_exc_env/spark_env/bin/python' #/usr/bin/python3
os.environ['PYSPARK_DRIVER_PYTHON'] = '/usr/bin/python3'

In [20]:
# didn't create connection due to the 
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()

/home/ekubay/Data-Engineering_text-to-speech_data-collection/env/lib/python3.8/site-packages/pyspark/bin/spark-class: line 71: ~/Data-Engineering_text-to-speech_data-collection/env/lib/python3.8/site-packages/bin/java: No such file or directory
/home/ekubay/Data-Engineering_text-to-speech_data-collection/env/lib/python3.8/site-packages/pyspark/bin/spark-class: line 96: CMD: bad array subscript


RuntimeError: Java gateway process exited before sending its port number

In [ ]:

#current path
path = '../data/Amharic News Dataset.csv'



In [ ]:
# reading the data 
df_pyspark = spark.read.csv(path,header=True,inferSchema=True)

In [17]:
# streaming text
#!pip install kafka-python


In [ ]:
# streaming text
from kafka import KafkaConsumer,KafkaProducer,KafkaAdminClient
from kafka.admin import NewTopic

